In [ ]:
from re import I
import torch
from torch import nn

import argparse
import json
import random

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score
from torch.utils import data
from HDF5Dataset import HDF5Dataset
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
from datetime import datetime
from skimage.metrics import structural_similarity as ssim
from scipy.spatial import distance
import pandas as pd
from skimage.util import img_as_float
import csv
import shutil

import random

from torch.nn import functional as F

from torch.autograd import Variable


# Merge fakes and train image folder

In [ ]:
train_dir = r"G:\My Drive\Projektarbeit_ResearchProject\datasets\BUS_classification\resized\train_val\260\500"
fake_dir = r"G:\My Drive\Projektarbeit_ResearchProject\datasets\BUS_classification\resized\fakes\StyleGAN2\500\200perc"
fakes_and_train_dir = r"G:\My Drive\Projektarbeit_ResearchProject\datasets\BUS_classification\resized\fakes_and_train\StyleGAN2\500\200perc"

os.makedirs(os.path.join(fakes_and_train_dir, "images"), exist_ok=True)
dest = os.path.join(fakes_and_train_dir, "images")

# copy train files
train_dir_imgs = os.listdir(os.path.join(train_dir, "images"))
for img in train_dir_imgs:
        full_file_name = os.path.join(os.path.join(train_dir, "images"), img)
        shutil.copy(full_file_name, dest)

# copy fake files
fake_dir_imgs = os.listdir(os.path.join(fake_dir, "images"))
for fake_img in fake_dir_imgs:
        full_file_name = os.path.join(os.path.join(fake_dir, "images"), fake_img)
        shutil.copy(full_file_name, dest)


# os.makedirs(os.path.join(fakes_and_train_dir, "images"), exist_ok=True)

# a = pd.read_csv(os.path.join)
# b = pd.read_csv("fileb.csv")

# Crop images

In [ ]:

img_path = r"G:\My Drive\Projektarbeit_ResearchProject\experiments\IVUS\IVUS_STYLEGAN2_DIFFAUG\IVUS_11_20_2\00000--cond-auto1-batch32-translation-cutout\fakes000840.png"


img = Image.open(img_path)
area = (0, 0, 8*256, 4*256)
cropped_img = img.crop(area)
cropped_img.show()

dataset_size = 42

output_folder = rf"C:\Users\Lenna\Desktop\Appendix\IVUS\StyleGAN2DiffAug\samples_for_appendix{dataset_size}.png"

cropped_img.save(output_folder)

# Plot metrics over training epochs - BUS

## Plots

In [ ]:
fig = plt.figure()
fig.set_size_inches(8.0, 5.0)
ax = fig.add_subplot(111, label="1")
ax2 = fig.add_subplot(111, label="2", frame_on=False)
ax3 = fig.add_subplot(111, label="3", frame_on=False)
ax4 = fig.add_subplot(111, label="4", frame_on=False)


ax.plot(x_axis_500, metric_df_mean_500, marker='.', color='C0')
ax2.plot(x_axis_250, metric_df_mean_250, marker='.', color='C1')
ax3.plot(x_axis_100, metric_df_mean_100, marker='.', color='C2')
ax4.plot(x_axis_50, metric_df_mean_50, marker='.', color='C3')

ax.fill_between(x_axis_500, metric_df_max_500, metric_df_min_500, alpha=0.1, edgecolor='C0', facecolor='C0')
ax2.fill_between(x_axis_250, metric_df_max_250, metric_df_min_250, alpha=0.1, edgecolor='C1', facecolor='C1')
ax3.fill_between(x_axis_100, metric_df_max_100, metric_df_min_100, alpha=0.1, edgecolor='C2', facecolor='C2')
ax4.fill_between(x_axis_50, metric_df_max_50, metric_df_min_50, alpha=0.1, edgecolor='C3', facecolor='C3')

ax.set(xlabel="Number of images (in thousand)", ylabel="FID score")
ax.set_ylim([50,450])
ax2.set_ylim([50,450])
ax3.set_ylim([50,450])
ax4.set_ylim([50,450])

ax.legend(["500"], loc=(0.55, 0.625))
ax2.legend(["250"], loc=(0.55, 0.525))
ax3.legend(["100"], loc=(0.7, 0.625))
ax4.legend(["50"], loc=(0.7, 0.525))

ax.grid()
ax2.grid()
#ax3.grid()


ticks = [i*60*2 for i in range(9)]
ax.xaxis.set_ticks(ticks)
ax2.xaxis.set_ticks(ticks)
ax3.xaxis.set_ticks(ticks)
ax4.xaxis.set_ticks(ticks)

ax.yaxis.set_ticks([400, 200, 150, 100, 50])
ax2.axis('off')
ax3.axis('off')
ax4.axis('off')
plt.show()

## Plot

In [ ]:
fig = plt.figure()
fig.set_size_inches(8.0, 5.0)
ax = fig.add_subplot(111, label="1")
ax2 = fig.add_subplot(111, label="2", frame_on=False)
ax3 = fig.add_subplot(111, label="3", frame_on=False)
# ax4 = fig.add_subplot(111, label="4", frame_on=False)
# ax5 = fig.add_subplot(111, label="4", frame_on=False)
# ax6 = fig.add_subplot(111, label="4", frame_on=False)


ax.plot(x_axis_100, metric_df_100, marker='.', color='C0')
ax2.plot(x_axis_50, metric_df_50, marker='.', color='C1')
ax3.plot(x_axis_25, metric_df_25, marker='.', color='C2')
# ax4.plot(x_axis_10, metric_df_10, marker='.', color='C3')
# ax5.plot(x_axis_5, metric_df_5, marker='.', color='C4')
# ax6.plot(x_axis_2, metric_df_2, marker='.', color='C5')


ax.set(xlabel="Number of images (in thousand)", ylabel="FID score")
ax.set_ylim([0,300])
ax2.set_ylim([0,300])
ax3.set_ylim([0,300])
# ax4.set_ylim([0,450])
# ax5.set_ylim([0,450])
# ax6.set_ylim([0,450])

ax.legend(["1366"], loc=(0.55, 0.725))
ax2.legend(["662"], loc=(0.7, 0.725))
ax3.legend(["377"], loc=(0.85, 0.725))
# ax4.legend(["10"], loc=(0.7, 0.725))
# ax5.legend(["5"], loc=(0.85, 0.825))
# ax6.legend(["2"], loc=(0.85, 0.725))

ax.grid()
ax2.grid()
#ax3.grid()


ticks = [i*60*2 for i in range(15)]
ax.xaxis.set_ticks(ticks)
ax2.xaxis.set_ticks(ticks)
ax3.xaxis.set_ticks(ticks)

ax.yaxis.set_ticks([300, 200, 150, 100, 50, 25])
ax2.axis('off')
ax3.axis('off')
# ax4.axis('off')
# ax5.axis('off')
# ax6.axis('off')
plt.show()

In [ ]:
fig = plt.figure()
fig.set_size_inches(8.0, 5.0)
ax4 = fig.add_subplot(111, label="4")
ax5 = fig.add_subplot(111, label="4", frame_on=False)
ax6 = fig.add_subplot(111, label="4", frame_on=False)

ax4.plot(x_axis_10, metric_df_10, marker='.', color='C3')
ax5.plot(x_axis_5, metric_df_5, marker='.', color='C4')
ax6.plot(x_axis_2, metric_df_2, marker='.', color='C5')

ax4.set_ylim([0,450])
ax5.set_ylim([0,450])
ax6.set_ylim([0,450])

ax4.legend(["205"], loc=(0.55, 0.725))
ax5.legend(["104"], loc=(0.7, 0.725))
ax6.legend(["42"], loc=(0.85, 0.725))

ax4.grid()
ax5.grid()
ax6.grid()

ticks = [i*60*2 for i in range(10)]
ax4.xaxis.set_ticks(ticks)
ax5.xaxis.set_ticks(ticks)
ax6.xaxis.set_ticks(ticks)

ax4.set(xlabel="Number of images (in thousand)", ylabel="FID score")

ax4.yaxis.set_ticks([400, 200, 150, 100, 50, 25])

ax5.axis('off')
ax6.axis('off')

plt.show()

# CAMUS

## SPADE

### Plot

In [ ]:
fig = plt.figure()
fig.set_size_inches(8.0, 5.0)
ax = fig.add_subplot(111, label="1")
ax2 = fig.add_subplot(111, label="2", frame_on=False)
ax3 = fig.add_subplot(111, label="3", frame_on=False)
# ax4 = fig.add_subplot(111, label="4", frame_on=False)
# ax5 = fig.add_subplot(111, label="4", frame_on=False)
# ax6 = fig.add_subplot(111, label="4", frame_on=False)



x_axis = [i*1280 for i in range(14)]

ax.plot(x_axis, metric_df_1280_SPADE, marker='.', color='C0')
ax2.plot(x_axis, metric_df_640_SPADE, marker='.', color='C1')
ax3.plot(x_axis, metric_df_320_SPADE, marker='.', color='C2')
# ax4.plot(x_axis_10, metric_df_10, marker='.', color='C3')
# ax5.plot(x_axis_5, metric_df_5, marker='.', color='C4')
# ax6.plot(x_axis_2, metric_df_2, marker='.', color='C5')


ax.set(xlabel="Number of images", ylabel="FID score")
ax.set_ylim([0,350])
ax2.set_ylim([0,350])
ax3.set_ylim([0,350])
# ax4.set_ylim([0,450])
# ax5.set_ylim([0,450])
# ax6.set_ylim([0,450])

ax.legend(["1280"], loc=(0.55, 0.725))
ax2.legend(["640"], loc=(0.7, 0.725))
ax3.legend(["320"], loc=(0.85, 0.725))
# ax4.legend(["10"], loc=(0.7, 0.725))
# ax5.legend(["5"], loc=(0.85, 0.825))
# ax6.legend(["2"], loc=(0.85, 0.725))

ax.grid()
ax2.grid()
#ax3.grid()

x_axis_ticks = [i*2*1280 for i in range(7)]

ax.xaxis.set_ticks(x_axis_ticks)
ax2.xaxis.set_ticks(x_axis_ticks)
ax3.xaxis.set_ticks(x_axis_ticks)


ax.yaxis.set_ticks([ 300, 200, 150, 100, 50, 25])
ax2.axis('off')
ax3.axis('off')
# ax4.axis('off')
# ax5.axis('off')
# ax6.axis('off')
plt.show()

In [ ]:
fig = plt.figure()
fig.set_size_inches(8.0, 5.0)
ax4 = fig.add_subplot(111, label="4")
ax5 = fig.add_subplot(111, label="4", frame_on=False)
ax6 = fig.add_subplot(111, label="4", frame_on=False)
ax7 = fig.add_subplot(111, label="4", frame_on=False)

x_axis = [i*160 for i in range(50)]


ax4.plot(x_axis, metric_df_160_SPADE, marker='.', color='C3')
ax5.plot(x_axis, metric_df_80_SPADE, marker='.', color='C4')
ax6.plot(x_axis, metric_df_40_SPADE, marker='.', color='C5')
ax7.plot(x_axis, metric_df_20_SPADE, marker='.', color='C6')


ax4.set(xlabel="Number of images", ylabel="FID score")
ax4.set_ylim([0,600])
ax5.set_ylim([0,600])
ax6.set_ylim([0,600])
ax7.set_ylim([0,600])


ax4.legend(["160"], loc=(0.35, 0.725))
ax5.legend(["80"], loc=(0.5, 0.725))
ax6.legend(["40"], loc=(0.65, 0.725))
ax7.legend(["20"], loc=(0.8, 0.725))


ax4.grid()
# ax2.grid()
# ax3.grid()

x_axis_ticks = [i*160*4 for i in range(13)]

ax4.xaxis.set_ticks(x_axis_ticks)
ax5.xaxis.set_ticks(x_axis_ticks)
ax6.xaxis.set_ticks(x_axis_ticks)
ax7.xaxis.set_ticks(x_axis_ticks)


ax4.yaxis.set_ticks([ 600, 400, 150, 100, 50, 25])
ax5.axis('off')
ax6.axis('off')
ax7.axis('off')
plt.show()